In [1]:
import restoration as res

c:\Users\Salvos\anaconda3\envs\multi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Salvos\anaconda3\envs\multi\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QFileDialog
import cv2
import os
from PyQt6.QtCore import QTimer  # Importa QTimer per la gestione del tempo

class MyApp(QWidget):
    def __init__(self):
        super().__init__()
        
        self.adjustSize()
        self.layout = QVBoxLayout()
        self.label = QLabel("")
        self.layout.addWidget(self.label)

        # Pulsante per importare il video
        self.videoImportButton = QPushButton("Importa Video")
        self.videoImportButton.clicked.connect(self.import_video)
        self.layout.addWidget(self.videoImportButton)

        # Pulsante per dividere in frame (inizialmente nascosto)
        self.frameSplitButton = QPushButton("Dividi in Frame")
        self.frameSplitButton.clicked.connect(self.split_frames)
        self.frameSplitButton.setVisible(False)
        self.layout.addWidget(self.frameSplitButton)
        
        # Pulsante per ulteriori operazioni dopo la divisione (inizialmente nascosto)
        self.highResButton = QPushButton("High Res")
        self.highResButton.clicked.connect(self.high_res)
        self.highResButton.setVisible(False)
        self.layout.addWidget(self.highResButton)

        self.denoiseButton = QPushButton("Denoising")
        self.denoiseButton.clicked.connect(self.denoise)
        self.denoiseButton.setVisible(False)
        self.layout.addWidget(self.denoiseButton)

        self.videoCreateButton = QPushButton("Genera video")
        self.videoCreateButton.clicked.connect(self.create_video)
        self.videoCreateButton.setVisible(False) 
        self.layout.addWidget(self.videoCreateButton)

        self.video_path = None
        self.video_name = None
        
        self.setLayout(self.layout)
        self.setWindowTitle("Gestione Video")

    def import_video(self):
        self.highResButton.setVisible(False)
        self.videoCreateButton.setVisible(False)
        self.denoiseButton.setVisible(False)

        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Seleziona un video", "", "Video Files (*.mp4 *.avi *.mov)")
        
        if file_path:
            self.video_path = file_path
            self.video_name = os.path.splitext(os.path.basename(file_path))[0]
            self.label.setText(f"Video importato: {os.path.basename(file_path)}")
            self.frameSplitButton.setVisible(True)  # Mostra il pulsante per dividere in frame
    
    def split_frames(self):
        if not self.video_path:
            self.label.setText("Nessun video selezionato!")
            return
        
        cap = cv2.VideoCapture(self.video_path)
        frame_count = 0
        output_folder = f"frames/{self.video_name}"
        os.makedirs(output_folder, exist_ok=True)
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.png")
            cv2.imwrite(frame_filename, frame)
            frame_count += 1
        
        cap.release()
        self.label.setText(f"Video diviso in {frame_count} frame salvati in '{output_folder}'")
        self.frameSplitButton.setVisible(False)
        self.highResButton.setVisible(True)
        self.denoiseButton.setVisible(True)
    
    def high_res(self):
        self.videoImportButton.setVisible(False)
        self.denoiseButton.setVisible(False)
        self.label.setText(f"Funzione High Res in esecuzione...")
        QTimer.singleShot(1000, self.apply_high_res)  

    def apply_high_res(self):
        res.hight_res(self.label, folder_frames=f"frames/{self.video_name}", save_dir= f'results/swinir_{self.video_name}')
        self.label.setText("Funzione High Res completata!")

        self.videoImportButton.setVisible(True)
        self.videoCreateButton.setVisible(True)
        self.denoiseButton.setVisible(True)
        self.highResButton.setVisible(False)
        
    def denoise(self):
        self.videoImportButton.setVisible(False)
        self.highResButton.setVisible(False)

        self.label.setText(f"Funzione Denoising in esecuzione...")
        res.denoise(self.label, folder_frames=f"frames/{self.video_name}", save_dir= f'results/swinir_{self.video_name}')
        self.label.setText(f"Funzione Denoising completata!")

        self.videoImportButton.setVisible(True)
        self.videoCreateButton.setVisible(True)
        self.highResButton.setVisible(False)

    def create_video(self):
        self.label.setText("Creazione video ...")
        folder=f'results/swinir_{self.video_name}'
        res.reconstruct_video_from_frames(frame_dir=folder, video_name=self.video_name)
        self.label.setText("Creazione video completata!")
        self.videoCreateButton.setVisible(False)  



if __name__ == "__main__":
    app = QApplication([])
    window = MyApp()
    window.show()
    app.exec()
